In [1]:
#import requests
#import json
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import RouteBuilder
from IPython.core.display import display, HTML
import polyline
%matplotlib inline

## Data

In [2]:
stores = pd.read_csv("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\store_coords.txt", names=["lat", "lon"])
custs_o = pd.read_csv("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\cust_coords.txt", names=["lat", "lon"])

In [3]:
depot = custs_o.iloc[0]

# random data

In [56]:
np.random.seed(9)
custX1 = np.random.uniform(-78.88, -78.67, 32)
custY1 = np.random.uniform(42.88, 43.04, 32)
custX2 = np.random.uniform(-78.85, -78.7, 8)
custY2 = np.random.uniform(42.8, 42.88, 8)

In [57]:
custs = np.concatenate(([depot], np.transpose([custY1, custX1]), np.transpose([custY2, custX2])))

In [58]:
custs = pd.DataFrame(custs, columns=["lat", "lon"])

In [59]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\cust_coords_rand9.txt", 'w') as f:
    for i in range(len(custs)):
        f.write(str(custs.iloc[i, 0]) + ", " + str(custs.iloc[i, 1]) + "\n")

## Plots

In [ ]:
m = folium.Map(location=(42.93, -78.79), zoom_start=11, tiles=None)
base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
base_map.add_to(m)
fg = folium.FeatureGroup(name="stores", overlay=False, show=True)
for i in range(len(stores)):
    folium.CircleMarker(location=tuple(stores.iloc[i]), color="red", radius=4).add_to(fg)
fg.add_to(m)
fg1 = folium.FeatureGroup(name="custs", overlay=True, show=True)
for i in range(len(custs)):
    folium.CircleMarker(location=tuple(custs.iloc[i]), color="orange", radius=3.5).add_to(fg1)
fg1.add_to(m)
m

## Routing distance

In [5]:
rb = RouteBuilder.RouteBuilder()
ss_dists_lst = []
for i in range(len(stores)):
    ss_dists_lst.append([])
    for j in range(i):
        ss_dists_lst[i].append(rb.makeRequest1(stores.iloc[i], stores.iloc[j]))

In [6]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\ss_dists_new.txt", 'w') as f:
    for x in ss_dists_lst:
        f.write(' '.join([str(y) for y in x]) + "\n")

In [61]:
rb = RouteBuilder.RouteBuilder()
cc_dists_lst = []
for i in range(len(custs)):
    cc_dists_lst.append([])
    for j in range(i):
        cc_dists_lst[i].append(rb.makeRequest1(custs.iloc[i], custs.iloc[j]))

In [62]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\cc_dists_rand3.txt", 'w') as f:
    for x in cc_dists_lst:
        f.write(' '.join([str(y) for y in x]) + "\n")

In [63]:
rb = RouteBuilder.RouteBuilder()
sc_dists_lst = []
for i in range(len(stores)):
    sc_dists_lst.append([])
    for j in range(len(custs)):
        sc_dists_lst[i].append(rb.makeRequest1(stores.iloc[i], custs.iloc[j]))

In [64]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\sc_dists_rand3.txt", 'w') as f:
    for x in sc_dists_lst:
        f.write(' '.join([str(y) for y in x]) + "\n")

In [23]:
for k in range(7, 10):
    np.random.seed(k)
    custX1 = np.random.uniform(-78.88, -78.67, 32)
    custY1 = np.random.uniform(42.88, 43.04, 32)
    custX2 = np.random.uniform(-78.85, -78.7, 8)
    custY2 = np.random.uniform(42.8, 42.88, 8)
    custs = np.concatenate(([depot], np.transpose([custY1, custX1]), np.transpose([custY2, custX2])))
    custs = pd.DataFrame(custs, columns=["lat", "lon"])
    rb = RouteBuilder.RouteBuilder()
    cc_dists_lst = []
    for i in range(len(custs)):
        cc_dists_lst.append([])
        for j in range(i):
            cc_dists_lst[i].append(rb.makeRequest1(custs.iloc[i], custs.iloc[j]))
    with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\cc_dists_rand%d.txt" % k, 'w') as f:
        f.write(str(len(custs)) + "\n")
        for x in cc_dists_lst:
            f.write(' '.join([str(y) for y in x]) + "\n")
    rb = RouteBuilder.RouteBuilder()
    sc_dists_lst = []
    for i in range(len(stores)):
        sc_dists_lst.append([])
        for j in range(len(custs)):
            sc_dists_lst[i].append(rb.makeRequest1(stores.iloc[i], custs.iloc[j]))
    with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\sc_dists_rand%d.txt" % k, 'w') as f:
        for x in sc_dists_lst:
            f.write(' '.join([str(y) for y in x]) + "\n")
    sc_routs = {}
    for i in range(len(stores)):
        for j in range(len(custs)):
            rt = rb.makeRequest(stores.iloc[i], custs.iloc[j])
            sc_routs[(len(custs) + i, j)] = polyline.decode(rt['geometry'])
    cc_routs = {}
    for i in range(len(custs)):
        for j in range(i):
            rt = rb.makeRequest(custs.iloc[i], custs.iloc[j])
            cc_routs[(i, j)] = polyline.decode(rt['geometry'])
    with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\sc_routs_rand%d.txt" % k, 'w') as f:
        for key in sc_routs.keys():
            f.write(str(key) + str(sc_routs[key]) + "\n")
    with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\cc_routs_rand%d.txt" % k, 'w') as f:
        for key in cc_routs.keys():
            f.write(str(key) + str(cc_routs[key]) + "\n")

# Routs (for plotting)

In [4]:
rb = RouteBuilder.RouteBuilder()
ss_routs = {}
rt_tmp = {}
for i in range(len(stores)):
    for j in range(i):
        rt = rb.makeRequest(stores.iloc[i], stores.iloc[j])
        rt_tmp[(len(custs) + i, len(custs) + j)] = rt
        ss_routs[(len(custs) + i, len(custs) + j)] = polyline.decode(rt['geometry'])

In [6]:
sc_routs = {}
for i in range(len(stores)):
    for j in range(len(custs)):
        rt = rb.makeRequest(stores.iloc[i], custs.iloc[j])
        sc_routs[(len(custs) + i, j)] = polyline.decode(rt['geometry'])

In [9]:
cc_routs = {}
for i in range(len(custs)):
    for j in range(i):
        rt = rb.makeRequest(custs.iloc[i], custs.iloc[j])
        cc_routs[(i, j)] = polyline.decode(rt['geometry'])

In [5]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\ss_routs_new.txt", 'w') as f:
    for key in ss_routs.keys():
        f.write(str(key) + str(ss_routs[key]) + "\n")

In [7]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\sc_routs_new.txt", 'w') as f:
    for key in sc_routs.keys():
        f.write(str(key) + str(sc_routs[key]) + "\n")

In [10]:
with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\cc_routs.txt", 'w') as f:
    for key in cc_routs.keys():
        f.write(str(key) + str(cc_routs[key]) + "\n")

In [3]:
for k in range(1, 10):
    np.random.seed(k)
    custX1 = np.random.uniform(-78.88, -78.67, 32)
    custY1 = np.random.uniform(42.88, 43.04, 32)
    custX2 = np.random.uniform(-78.85, -78.7, 8)
    custY2 = np.random.uniform(42.8, 42.88, 8)
    custs = np.concatenate((np.transpose([custY1, custX1]), np.transpose([custY2, custX2])))
    custs = pd.DataFrame(custs, columns=["lat", "lon"])
    rb = RouteBuilder.RouteBuilder()
    sc_routs = {}
    for i in range(len(stores)):
        for j in range(len(custs)):
            rt = rb.makeRequest(stores.iloc[i], custs.iloc[j])
            sc_routs[(len(custs) + i, j)] = polyline.decode(rt['geometry'])
    cc_routs = {}
    for i in range(len(custs)):
        for j in range(i):
            rt = rb.makeRequest(custs.iloc[i], custs.iloc[j])
            cc_routs[(i, j)] = polyline.decode(rt['geometry'])
    with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\sc_routs_rand%d.txt" % k, 'w') as f:
        for key in sc_routs.keys():
            f.write(str(key) + str(sc_routs[key]) + "\n")
    with open("D:\\Study\\Ph.D\\Projects\\Bilevel Optimization\\data\\Buffalo\\cc_routs_rand%d.txt" % k, 'w') as f:
        for key in cc_routs.keys():
            f.write(str(key) + str(cc_routs[key]) + "\n")

KeyboardInterrupt: 